In [1]:
print("ok")

ok


In [2]:
%pwd

'c:\\Users\\shyam\\OneDrive\\Documents\\GitHub\\AI-Bot\\research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\shyam\\OneDrive\\Documents\\GitHub\\AI-Bot'

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [7]:
extracted_data=load_pdf_file(data='Data/')

In [8]:
#extracted_data

In [9]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [34]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 37362


In [11]:
#text_chunks

In [12]:
#%pip uninstall sentence-transformers huggingface-hub -y
#%pip install sentence-transformers huggingface-hub --upgrade

In [13]:
#%pip install -U langchain-huggingface


In [14]:
from langchain_huggingface import HuggingFaceEmbeddings


In [15]:
from langchain_huggingface import HuggingFaceEmbeddings
import os

def download_hugging_face_embeddings():
    local_model_path = "C:/Users/shyam/models/all-MiniLM-L6-V2"
    
    if not os.path.exists(local_model_path):
        raise FileNotFoundError(f"Model path does not exist: {local_model_path}")
    
    print(f"Loading model from: {local_model_path}")
    embeddings = HuggingFaceEmbeddings(model_name=local_model_path)
    print("Model loaded successfully.")
    
    return embeddings

embeddings = download_hugging_face_embeddings()


Loading model from: C:/Users/shyam/models/all-MiniLM-L6-V2


c:\Users\shyam\OneDrive\Documents\GitHub\AI-Bot\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Model loaded successfully.


In [16]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [17]:
#query_result

In [83]:
from dotenv import load_dotenv
load_dotenv()

True

In [36]:
# Import the Pinecone library
from pinecone import Pinecone
import os

# Initialize a Pinecone client with your API key
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')

In [28]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medicalbot"


pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
)

{
    "name": "medicalbot",
    "metric": "cosine",
    "host": "medicalbot-hbpacab.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [29]:
from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

In [38]:
from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings, 
)

In [39]:
docsearch

In [40]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [41]:
retrieved_docs = retriever.invoke("What is Acne?")

In [42]:
retrieved_docs

[Document(id='c9854819-8252-4f54-b7dd-e1e59a9a0808', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Gale Encyclopedia of Medicine Vol. 1 (A-B).pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='3162659e-9aec-481c-ae72-36fcc3dcee25', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 38.0, 'page_label': '39', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Gale Encyclopedia of Medicine Vol. 1 (A-B).pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed.(Photograph by Biophoto Associ-\nate

In [85]:
import os 
GROQ_API_KEY=os.environ.get('GROQ_API_KEY')
os.environ["GROQ_API_KEY"] = GROQ_API_KEY

In [88]:
import os

from groq import Groq

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Explain the importance of fast language models",
        }
    ],
    model="llama-3.3-70b-versatile",
)

print(chat_completion.choices[0].message.content)

Fast language models are revolutionizing the field of natural language processing (NLP) and have numerous applications in various industries. The importance of fast language models can be attributed to the following factors:

1. **Speed and Efficiency**: Fast language models can process large amounts of text data at incredible speeds, making them ideal for applications where response time is crucial, such as chatbots, virtual assistants, and real-time sentiment analysis.
2. **Improved User Experience**: Fast language models enable developers to create responsive and interactive applications that can engage users in real-time, leading to enhanced user experiences and increased satisfaction.
3. **Scalability**: Fast language models can handle a massive volume of requests, making them perfect for large-scale applications, such as language translation, text summarization, and sentiment analysis, where speed and efficiency are essential.
4. **Real-time Applications**: Fast language models a

In [92]:
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

llm = ChatGroq(model_name="llama-3.3-70b-versatile", temperature=0.7)

chain = prompt | llm

In [109]:
from flask import request
def chat():
    msg = request.form["msg"]
    input = msg
    print(input)
    response = chain.invoke({"input": msg, "context": ""})
    response_terms = set(response.lower().split())
    retriever_terms = set(retriever.keys())
    if not response_terms.issubset(retriever_terms):
        response = "Yeh Meri Expertise Nahi Hai, so I Am Out."
    print("Response : ", response)
    return str(response)

In [110]:
response = chain.invoke({"input": "what is Acromegaly and gigantism?", "context": ""})
print(response)

content='Acromegaly and gigantism are rare hormonal disorders caused by an excess of growth hormone (GH) in the body. Gigantism occurs when the excess GH is produced before the bones have fully grown, typically during childhood, leading to excessive growth and tall stature. Acromegaly, on the other hand, occurs when the excess GH is produced after normal growth has stopped, typically in adulthood, leading to enlarged hands, feet, and facial features.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 94, 'prompt_tokens': 95, 'total_tokens': 189, 'completion_time': 0.341818182, 'prompt_time': 0.005043739, 'queue_time': 0.058719501, 'total_time': 0.346861921}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_9a8b91ba77', 'finish_reason': 'stop', 'logprobs': None} id='run-564c9eb9-5899-452e-94c2-1b8bd93e8228-0' usage_metadata={'input_tokens': 95, 'output_tokens': 94, 'total_tokens': 189}


In [95]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Answer the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
)

In [111]:
response = chain.invoke({"input": "What is stars?", "context": ""})
print(response)

content='Stars are massive, luminous balls of gas that are held together by their own gravity. They are primarily composed of hydrogen and helium, and are the source of light and heat for many planets, including Earth. The Sun is an example of a star, and there are billions of other stars in the universe.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 63, 'prompt_tokens': 88, 'total_tokens': 151, 'completion_time': 0.229090909, 'prompt_time': 0.005905042, 'queue_time': 0.060029678, 'total_time': 0.234995951}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_9a8b91ba77', 'finish_reason': 'stop', 'logprobs': None} id='run-81e05692-c060-4f3f-a421-96f414c4ee9c-0' usage_metadata={'input_tokens': 88, 'output_tokens': 63, 'total_tokens': 151}
